In [35]:
import pandas as pd
import numpy as np

In [36]:
qwen_7B_init = pd.read_csv('inference_results_qwen_7B_init.csv')
qwen_7B_sft = pd.read_csv('inference_results_qwen_7B_sft.csv')
qwen_14B_init = pd.read_csv('inference_results_qwen_14B_init.csv')
qwen_14B_sft = pd.read_csv('inference_results_qwen_14B_sft.csv')


In [37]:
# 比较正确率,召回率,查准率, F1值
acc_set = []
recall_set = []
precision_set = []
f1_set = []

data_set = [qwen_7B_init, qwen_7B_sft, qwen_14B_init, qwen_14B_sft]
res_set = []
for data in data_set:
    res = []
    for i in range(300):
        label = data['Expected Output'][i]
        pred = data['Generated Output'][i]
        res.append([label, pred])
    res_set.append(res)
    

# 计算正确率
for i in range(4):
    acc = 0
    for j in range(300):
        if res_set[i][j][0] == res_set[i][j][1]:
            acc += 1
    acc_set.append(acc / 300)

# # 计算召回率
# for i in range(4):
#     tp = 0
#     fn = 0
#     for j in range(300):
#         if res_set[i][j][0] == res_set[i][j][1] and res_set[i][j][0] == 1:
#             tp += 1
#         if res_set[i][j][0] == 1:
#             fn += 1
#     if tp + fn == 0:
#         recall_set.append('-')
#     else:
#         recall_set.append(tp / (tp + fn))

# # 计算查准率
# for i in range(4):
#     tp = 0
#     fp = 0
#     for j in range(300):
#         if res_set[i][j][0] == res_set[i][j][1] and res_set[i][j][0] == 1:
#             tp += 1
#         if res_set[i][j][1] == 1:
#             fp += 1
#     if tp + fp == 0:
#         precision_set.append('-')
#     else:
#         precision_set.append(tp / (tp + fp))

# # 计算F1值
# for i in range(4):
#     if precision_set[i] == '-' or recall_set[i] == '-':
#         f1_set.append('-')
#     else:
#         f1 = 2 * precision_set[i] * recall_set[i] / (precision_set[i] + recall_set[i])
#         if precision_set[i] == '-' or recall_set[i] == '-':
#             f1_set.append('-')
#         else:
#             f1_set.append(f1)
print('7B_init:', acc_set[0])
print('7B_sft:', acc_set[1])
print('14B_init:', acc_set[2])
print('14B_sft:', acc_set[3])
# print(recall_set)
# print(precision_set)
# print(f1_set)


7B_init 0.25
7B_sft 0.5433333333333333
14B_init 0.38
14B_sft 0.49
